### 함수들

In [1]:
# 연도별로 시총 top 1000에 드는 회사만을 데이터셋에 포함시킨다
def AnnualCap(raw_data, start_year, end_year, num_top=1000):
    raw_data['date'] = pd.to_datetime(raw_data['date'])
    df_top = pd.DataFrame()
    
    for year in range(start_year, end_year+1): # 매년
        # 각 연도에 해당하는 데이터만을 추출
        temp = raw_data[raw_data['date'].dt.year == year]
        
        # 회사(permno)별로 시가총액(me)의 평균을 구한 후, 시총이 큰 순서대로 내림차순 정렬
        # 내림차순 정렬된 상태에서 상위 num_top개의 회사를 추출하여 리스트로 만들기
        top_list = temp.groupby('permno').mean()[['me']].reset_index().\
                    sort_values(by='me', ascending=False)[:num_top]['permno'].tolist()
        # temp에서 상위 num_top개의 회사만을 포함하는 데이터를 추출 후 이를 매년 반복한 데이터와 concat
        df_top = pd.concat([df_top, temp[temp['permno'].isin(top_list)]], axis=0)

    return df_top

In [2]:
# Inf, -Inf 있는 경우 0으로 대체
def ind_clean_data(raw_data, scaling_features):
    raw_data[scaling_features] = raw_data[scaling_features].astype('float32')
    for col in scaling_features:
        is_inf = (raw_data[col] == float("inf")) | (raw_data[col] == float("-inf"))
        res = sum(is_inf)
        if res != 0:
            raw_data.loc[is_inf,col] = 0

In [3]:
from sklearn.preprocessing import MinMaxScaler
# min max scaling을 하기 위한 함수
def my_scaler(scaling_features, train_data, test_data):
    '''
    scaling_features: min max scaling을 적용할 변수들 리스트
    train data로 fit 한 후 
    train data와 test data에 scaling을 적용(transform)한다.
    '''
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    scaler.fit(train_data[scaling_features])
    
    train_data[scaling_features] = scaler.transform(train_data[scaling_features])
    test_data[scaling_features] = scaler.transform(test_data[scaling_features])

In [4]:
def make_stationary(macro_data):
    d_mac = macro_data.copy()
    for col in d_mac.iloc[:,1:].columns:
        d_mac[f'd_{col}'] = (d_mac[col] - d_mac[col].shift(1).bfill())/d_mac[col]
        d_mac.drop(col, axis=1, inplace=True)
    return d_mac

In [5]:
# 데이터프레임으로 되어있는 macro 데이터를 넘파이 배열로 변환 후 npz 파일로 저장하는 함수
def macro_to_npz(d_mac, filename):
    mac_date = d_mac['sasdate'].astype(str).to_numpy()
    mac_variables = d_mac.columns[1:].to_numpy()
    d_mac_np = d_mac.iloc[:,1:].to_numpy()
    np.savez(f'{filename}', date=mac_date, variable=mac_variables, data=d_mac_np)

In [6]:
def macro_clean_data(d_mac_data):
    d_mac_data.iloc[:,1:] = d_mac_data.iloc[:,1:].astype('float32')
    for col in d_mac_data.iloc[:,1:].columns:
        is_inf = (d_mac_data[col] == float("inf")) | (d_mac_data[col] == float("-inf"))
        res = sum(is_inf)
        if res != 0:
            d_mac_data.loc[is_inf,col] = 0
    d_mac_data.iloc[:,1:] = d_mac_data.iloc[:,1:].fillna(0)

In [7]:
# 모델 refit 하는 것처럼 scaling도 같은 방식으로
def re_scaling(df, scaling_features, train_start=1996, train_end=2016, test_size=5):
    tmp = df.copy()
    tmp['year'] = tmp['sasdate'].dt.year
    # scaling test set
    new_test = pd.DataFrame()
    for i in range(test_size):
        start_year = train_start
        end_year = train_end + i

        train = tmp[(tmp['year']>=start_year) & (tmp['year']<=end_year)]
        test = tmp[tmp['year']==end_year+1]

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(train[scaling_features])
        test[scaling_features] = scaler.transform(test[scaling_features])
        new_test = pd.concat([new_test, test], axis=0)
    
    # scaling train set
    full_train = tmp[(tmp['year']>=train_start) & (tmp['year']<=train_end)]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(full_train[scaling_features])
    full_train[scaling_features] = scaler.transform(full_train[scaling_features])

    return full_train, new_test

# firm.npz

In [8]:
import pandas as pd
import numpy as np

In [3]:
raw1990 = pd.read_csv('./data/raw_data/firm-specific/chars60_raw_1990s.csv')
raw2000 = pd.read_csv('./data/raw_data/firm-specific/chars60_raw_2000s.csv')
raw2010 = pd.read_csv('./data/raw_data/firm-specific/chars60_raw_2010s.csv')
raw2020 = pd.read_csv('./data/raw_data/firm-specific/chars60_raw_2020s.csv')

raw = pd.concat([raw1990, raw2000, raw2010, raw2020], axis=0)
raw['date'] = pd.to_datetime(raw['date'])

In [6]:
raw_top = AnnualCap(raw, 1996, 2021, 1000)

In [9]:
scaling_features = ['adm',
       'bm_ia', 'herf', 'hire', 'me_ia', 'ill', 'maxret', 'mom12m', 'mom1m',
       'mom36m', 'mom60m', 'mom6m', 'std_dolvol', 'me', 'dy', 'cinvest',
       'nincr', 'pscore', 'acc', 'bm', 'agr', 'alm', 'ato', 'cash', 'cashdebt',
       'cfp', 'chcsho', 'chpm', 'chtx', 'depr', 'ep', 'gma', 'grltnoa', 'lev',
       'lgr', 'ni', 'noa', 'op', 'pctacc', 'pm', 'rd_sale', 'rdm', 'rna',
       'roa', 'roe', 'rsup', 'sgr', 'sp']

ind_clean_data(raw_top, scaling_features)

scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(raw_top[scaling_features])
raw_top[scaling_features] = scaler.transform(raw_top[scaling_features])

In [121]:
import datetime as dt
class Individual_Preprocess:
    def __init__(self, raw_data, filename):
        self.filename = filename
        self.raw_data = raw_data
        self.start = "1996-01-01"
        self.end = "2021-12-31"

        self.make_same_length()
        self.ind_to_npz()
        
    def make_same_length(self):
        lis = pd.date_range(start=self.start, end=self.end, freq='M') 
        lis = [date_obj.strftime('%Y-%m-%d') for date_obj in lis]
        lis = pd.DataFrame({'date':lis})
        lis['date'] = pd.to_datetime(lis['date'])
        self._lis = lis

        unq_date = lis['date'].apply(lambda x: int(x.strftime('%Y%m%d')))
        self._unq_date = unq_date.to_numpy()

        raw_merged = pd.DataFrame()
        for perm in self.raw_data['permno'].unique():
            tmp = self.raw_data[self.raw_data['permno']==perm]
            tmp = tmp.sort_values(by='date')
            tmp_merged = pd.merge(tmp, lis, on=['date'], how='right')
            tmp_merged['permno'] = perm
            raw_merged = pd.concat([raw_merged, tmp_merged], axis=0)
        
        raw_merged = raw_merged.sort_values(by=['date','permno'])
        self._raw_merged = raw_merged
    
    def ind_to_npz(self):
        
        self._raw_merged = self._raw_merged.drop(['permno','ticker','gvkey','sic','exchcd',
                                                  'shrcd','ffi49','year','date'], axis=1)
        self._raw_merged = self._raw_merged.fillna(-99.99)
        
        num_dates = self._lis.shape[0]
        num_permno = self.raw_data['permno'].nunique()
        num_ind_features = self._raw_merged.shape[1] 
        print('num_ind_features',num_ind_features)
        
        raw_merged_np = self._raw_merged.to_numpy()
        raw_merged_np = raw_merged_np.reshape(num_dates, num_permno, num_ind_features)
        
        ind_variables = self._raw_merged.columns.to_numpy()
        
        np.savez(f'{self.filename}', date=self._unq_date, variable=ind_variables, data=raw_merged_np)

In [224]:
Individual_Preprocess(raw_top, 'firm')

num_ind_features 49


In [136]:
firm = np.load('firm.npz', allow_pickle=True)

# mac.npz

- macro 데이터도 firm-specific 데이터와 마찬가지로 변수마다 업데이트되는 주기가 다름
- 어떤 변수는 3달 후에 알 수 있고(lag3_vars), 어떤 변수는 2달 후에 알 수 있고(lag2_vars) 등등..
- 이런 변수별 업데이트 주기를 반영해야 함 
- 이때, 현재 시점에서 알 수 있는 가장 최근 값을 사용하도록 함

In [10]:
mac = pd.read_csv('./raw_data/macroeconomic/fred_macro_2022-05.csv')
mac['sasdate'] = pd.to_datetime(mac['sasdate'])

lag3_vars = ['CMRMTSPLx', 'HWI', 'HWIURATIO', 'ACOGNO', 'BUSINVx', 'ISRATIOx', 'NONREVSL', 'CONSPI', 
             'S&P div yield', 'DTCOLNVHFNM', 'DTCTHFNM']
lag5_vars = ['S&P PE ratio']
lag2_vars = mac.columns.drop(['sasdate']+lag3_vars+lag5_vars).tolist()

# 변수별로 업데이트 주기를 반영해서 lag 시키는 함수
def lag_vars(macro_data):
    macro_data[lag2_vars] = macro_data[lag2_vars].shift(2).bfill()
    macro_data[lag3_vars] = macro_data[lag3_vars].shift(3).bfill()
    macro_data[lag5_vars] = macro_data[lag5_vars].shift(5).bfill()

In [11]:
lag_vars(mac)
macro_clean_data(mac)

In [12]:
mac_all = mac[(mac['sasdate'] >= '1996-01-01') & (mac['sasdate'] <= '2022-12-31')]

In [13]:
# 비정상계열인 데이터를 정상계열로 만들어서 쓰기 위해 차분 후 나눔 (return 계산하듯이)
d_mac = make_stationary(mac)
macro_clean_data(d_mac)

In [14]:
# train test split
d_mac_train = d_mac[(d_mac['sasdate'] >= '1996-01-01') & (d_mac['sasdate'] <= '2016-12-31')]
d_mac_test = d_mac[(d_mac['sasdate'] >= '2017-01-01') & (d_mac['sasdate'] <= '2022-12-31')]

In [15]:
def my_scaler(scaling_features, train_data, test_data):
    '''
    scaling_features: min max scaling을 적용할 변수들 리스트
    train data로 fit 한 후 
    train data와 test data에 scaling을 적용(transform)한다.
    '''
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    scaler.fit(train_data[scaling_features])
    
    train_data[scaling_features] = scaler.transform(train_data[scaling_features])
    test_data[scaling_features] = scaler.transform(test_data[scaling_features])

In [16]:
d_mac_all = pd.concat([d_mac_train, d_mac_test], axis=0)

In [17]:
# 반드시 na가 없어야 함
d_mac_all.isna().sum().sum()

0

In [18]:
print(d_mac_all.shape)

(316, 128)


In [19]:
# npz 파일로 저장
macro_to_npz(d_mac_all, 'mac')

In [20]:
# npz 파일 불러오기
mac = np.load('mac.npz', allow_pickle=True)

# firm_mac.npz
---
- firm-specific 변수와 차분한 macro 변수를 합침

In [115]:
raw_top['month'] = raw_top['date'].dt.month

In [116]:
d_mac_all['year'] = d_mac_all['sasdate'].dt.year
d_mac_all['month'] = d_mac_all['sasdate'].dt.month

In [129]:
raw_d_mac_all = pd.merge(raw_top, d_mac_all, on=['year','month'], how='left')

In [130]:
raw_d_mac_all.drop(['month','sasdate'], axis=1, inplace=True)

In [131]:
Individual_Preprocess(raw_d_mac_all, 'firm_mac')

num_ind_features 176


In [132]:
firm_mac = np.load('firm_mac.npz', allow_pickle=True)

# firm_mac.csv

In [133]:
raw_d_mac_all.to_csv('firm_mac.csv', index=False)